# Ejercicio 9 historico de RV de cada activo de varios indices Yahoo

- Obtén el histórico (60 días) de cada activo de varios índices (usa los resultados del paso anterior)
- Recomendación: Yahoo Finance. Tiempo objetivo: 45 minutos. Índices: "%5EHSI", "%5EIBEX" cotizando en cada índice antes de hacer la consulta activo por activo. Recomendación: Yahoo Finance. Tiempo objetivo: 45 minutos

- Este ejercicio concatena los dos anteriores. 

       Paso 1: nos descarganos los indices y sus componentes
       Paso 2: pasamos los actvos por las funciones que nos permiten. 

In [2]:
import pandas as pd
import numpy as np
import requests
import re
from bs4 import BeautifulSoup
from datetime import datetime
import time
from tqdm import tqdm
import math
from rcurl import get_curl
from io import BytesIO
from time import mktime

In [3]:
def _get_crumbs_and_cookies(stock):
    """
    get crumb and cookies for historical data csv download from yahoo finance
    parameters: stock - short-handle identifier of the company 
    returns a tuple of header, crumb and cookie
    """
    
    url = 'https://finance.yahoo.com/quote/{}/history'.format(stock)
    
    with requests.session():
        
        header = {'Connection': 'keep-alive',
                   'Expires': '-1',
                   'Upgrade-Insecure-Requests': '1',
                   'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) \
                   AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.99 Safari/537.36'
                   }
        
        website = requests.get(url, headers=header)
        soup = BeautifulSoup(website.text, 'lxml')
        crumb = re.findall('"CrumbStore":{"crumb":"(.+?)"}', str(soup))

        return (header, crumb[0], website.cookies)

In [4]:
def convert_to_unix(date):
    """
    converts date to unix timestamp
    parameters: date - in format (yyyy-mm-dd)
    returns integer unix timestamp
    """
    datum = datetime.strptime(date, '%Y-%m-%d')
    
    return int(mktime(datum.timetuple()))

In [6]:
def load_csv_data(stock, interval='1d', day_begin='20-03-2018', day_end='20-06-2018'):
    """
    queries yahoo finance api to receive historical data in csv file format
        parameters: 
        stock - short-handle identifier of the company        
        interval - 1d, 1wk, 1mo - daily, weekly monthly data        
        day_begin - starting date for the historical data (format: dd-mm-yyyy)        
        day_end - final date of the data (format: dd-mm-yyyy)    
    returns a list of comma seperated value lines
    """
    
    day_begin_unix = convert_to_unix(day_begin)
    day_end_unix = convert_to_unix(day_end)
       
    header, crumb, cookies = _get_crumbs_and_cookies(stock)
    
    with requests.session():
        url = 'https://query1.finance.yahoo.com/v7/finance/download/'               '{stock}?period1={day_begin}&period2={day_end}&interval={interval}&events=history&crumb={crumb}'               .format(stock=stock, day_begin=day_begin_unix, day_end=day_end_unix, interval=interval, crumb=crumb)
                
        website = requests.get(url, headers=header, cookies=cookies)
       
        return website.text.split('\n')

In [7]:
def componentes_indice(tickers_de_indices):
    """esta funcion devuelve los componentes y divisa de cada indice.
    como parametro se debe inclur los tickers de Yahoo Finance de los indices
    en una tupla"""
    
    n=0
    for i in tickers_de_indices:

        Index=i
        
        #primero se obtiene la divisa
        url="https://es.finance.yahoo.com/quote/"+i+"/components/"
        soup  = requests.get(url)
        soup  = BeautifulSoup(soup.content, 'html.parser')       
        name_box = soup.find('span', attrs={'data-reactid': '4'})
        
        try:
            name = name_box.text.strip()
            divisa=name[len(name)-3:len(name)]
            print(i,divisa)
            
            # Reconstruimos la tabla. Tiene 6 columnas (símbolo, nombre de la empresa, último precio, cambio, cambio % y volumen.)
            # Nos interesa obtener el símbolo y el nombre de la empresa.
            # Ojo, queremos importar únicamente los activos que tengan cotización. 
            
            soup  = requests.get(url)
            soup  = BeautifulSoup(soup.content, 'html.parser')
            
            name_box = soup.find_all('tr')
            name_boxccc=name_box[1]
            name_boxccc = name_boxccc.get_text(strip=True)
            lista_indice=[]
            lista_activo=[]
            lista_divisa=[]
            lista_num_errores=[]
            lista_empresa=[]
            lista_resto=[]

            for i in range(1,len(name_box)-1):
                try:
                    lista_indice.append(Index)
                    name_boxccc=name_box[i]
                    name_boxccc = name_boxccc.get_text(strip=True)
                    name_boxccc=name_boxccc.split(sep='.')
                    lista_activo.append(name_boxccc[0]+'.'+name_boxccc[1][0:2])
                    lista_empresa.append(name_boxccc[1][2:5]+'.'+name_boxccc[2][0:len(name_boxccc[2])])
                    lista_resto.append(name_boxccc)
                    lista_divisa.append(divisa)

                except:
                    lista_num_errores.append(i)

            if len(lista_activo)==len(lista_divisa) & len(lista_activo)==len(lista_empresa) & len(lista_activo)==len(lista_resto):

                df=pd.DataFrame ({'indice':lista_indice,
                                'activo':lista_activo,
                                'divisa':lista_divisa,
                                'resto':lista_resto})
                df['posicion']=np.where(df['resto'].astype(str).str.contains('%', regex=False)==True,1,0)

                df=df[df['posicion']==1]
                df=df.drop(['resto','posicion'],1)

                if n==0:
                    dfacum=df
                else:
                    dfacum=pd.concat([dfacum, df], axis=0,sort=True)
                    dfacum.reset_index(drop=True)
                n=n+1

            else:
                print('El activo '+str(i)+'no se ha podido incorporar por inconsistencias en los datos importados')

        except:
            print('Para la referencia '+str(i)+' no se ha podido descargar la informacion')
    
    return dfacum

In [14]:
def set_dates(x,ventana):
    
    """setup inicial de fechas,
    esta funcion devuelve la fecha de inicio
    y fin en string y formato YYYY-MM-DD
    Parametros:
    x=start o end
    ventana: entero referido al numero de dias del periodo"""
    
    from datetime import timedelta
    from datetime import datetime
    hoy=datetime.now()
    if x=='end':
        fecha_fin = str(hoy.now())
        fecha_fin = fecha_fin[0:10]
        return (fecha_fin)
    if x=='start':
        fecha_inicial=hoy-timedelta(ventana)
        fecha_inicial = str(fecha_inicial)
        fecha_inicial = fecha_inicial[0:10]
        return (fecha_inicial)
    else:
        print('inputs incorrectos. Ver docstring de la funcion')

In [8]:
def historic_prices_series(stock_series,interval,fecha_inicial,fecha_fin):
    
    n=0
    
    for benchmark in stock_series:
        
        print("descargando "+benchmark)
        try:
            DF=load_csv_data(benchmark, interval=interval, day_begin=fecha_inicial, day_end=fecha_fin)
        except:
            print("No nos hemos podido descargar el indice")
            
        DF=pd.DataFrame(DF)
        DF = DF.iloc[:,0].str.split(",", n = 7, expand = True)
        DF.columns=DF.iloc[0,:]
        DF=DF.iloc[1:DF.shape[0],:]
        DF['stock']=str(benchmark)
        DF=DF.dropna()
        
        if n==0:
            dfacum=DF
        else:
            dfacum=agrega_dataframes(DF,dfacum)    
        n=n+1
        
    return dfacum

In [9]:
def agrega_dataframes(df,dfacum):
    """codigo que permite ir concatenando
    dataframes con estructura similar
    df:daframe a agregar
    dfacum: dataframe en donde se agregara"""
    
    dfacum=pd.concat([dfacum, df], axis=0,sort=True)
    dfacum.reset_index(drop=True)
    
    return dfacum

In [10]:
tickers_de_indices = ("%5EIBEX","%5EHSI")
ventana=60

In [11]:
componentes_indice = componentes_indice(tickers_de_indices)

%5EIBEX EUR
%5EHSI HKD


In [12]:
stock_series = componentes_indice['activo']
stock_series

0      BKT.MC
1     CLNX.MC
2      ELE.MC
3      ENG.MC
4      MTS.MC
5      REE.MC
6      VIS.MC
7     BBVA.MC
8     SGRE.MC
9      ACS.MC
10    CABK.MC
11     MAP.MC
12    NTGY.MC
13    AENA.MC
14     FER.MC
15     COL.MC
16     TL5.MC
17     IBE.MC
18     AMS.MC
19     ITX.MC
20     SAN.MC
21     TEF.MC
22     MEL.MC
23     ACX.MC
24     GRF.MC
25     ENC.MC
26     ANA.MC
27     MRL.MC
28     IAG.MC
29     SAB.MC
0     0267.HK
1     0011.HK
2     1299.HK
3     1044.HK
4     2628.HK
5     0003.HK
6     2319.HK
7     0012.HK
8     0883.HK
9     1038.HK
10    2318.HK
11    2018.HK
12    1997.HK
13    0101.HK
14    1928.HK
15    0002.HK
16    0669.HK
17    0016.HK
18    0941.HK
19    1398.HK
20    0386.HK
21    2388.HK
22    1109.HK
23    2007.HK
24    0006.HK
25    0083.HK
26    0288.HK
27    1093.HK
28    0700.HK
29    0027.HK
Name: activo, dtype: object

In [15]:
fecha_fin = set_dates('end',ventana)
fecha_inicial = set_dates('start',ventana)

print('la fecha de inicio es',fecha_inicial,' y la de fin es',fecha_fin)

la fecha de inicio es 2020-05-01  y la de fin es 2020-06-30


In [16]:
dfacum = historic_prices_series(stock_series,'1d',fecha_inicial,fecha_fin)

descargando BKT.MC
descargando CLNX.MC
descargando ELE.MC
descargando ENG.MC
descargando MTS.MC
descargando REE.MC
descargando VIS.MC
descargando BBVA.MC
descargando SGRE.MC
descargando ACS.MC
descargando CABK.MC
descargando MAP.MC
descargando NTGY.MC
descargando AENA.MC
descargando FER.MC
descargando COL.MC
descargando TL5.MC
descargando IBE.MC
descargando AMS.MC
descargando ITX.MC
descargando SAN.MC
descargando TEF.MC
descargando MEL.MC
descargando ACX.MC
descargando GRF.MC
descargando ENC.MC
descargando ANA.MC
descargando MRL.MC
descargando IAG.MC
descargando SAB.MC
descargando 0267.HK
descargando 0011.HK
descargando 1299.HK
descargando 1044.HK
descargando 2628.HK
descargando 0003.HK
descargando 2319.HK
descargando 0012.HK
descargando 0883.HK
descargando 1038.HK
descargando 2318.HK
descargando 2018.HK
descargando 1997.HK
descargando 0101.HK
descargando 1928.HK
descargando 0002.HK
descargando 0669.HK
descargando 0016.HK
descargando 0941.HK
descargando 1398.HK
descargando 0386.HK
desc

In [17]:
dfacum.head()

,Adj Close,Close,Date,High,Low,Open,Volume,stock
1,3.584000,3.584000,2020-05-04,3.681000,3.567000,3.681000,2262290,BKT.MC
2,3.520000,3.520000,2020-05-05,3.730000,3.520000,3.730000,2481819,BKT.MC
3,3.411000,3.411000,2020-05-06,3.546000,3.401000,3.546000,2268370,BKT.MC
4,3.444000,3.444000,2020-05-07,3.468000,3.351000,3.429000,2508451,BKT.MC
5,3.514000,3.514000,2020-05-08,3.538000,3.399000,3.470000,2366539,BKT.MC


### Otra manera de hacerlo

In [1]:
import urllib.request
import requests
from io import StringIO
from datetime import datetime,timedelta
import numpy as np
import pandas as pd
import time

In [2]:
tickers_de_indices=[ "%5EHSI", "%5EIBEX"]
ventana=60

In [4]:
def activos_indices (tickers_de_indices):
    
    tabla_activos = pd.DataFrame()
    
    for indice in tickers_de_indices:
        
        url="https://es.finance.yahoo.com/quote/"+indice+"/components/"
        
        #Extraemos la divisa en la que cotizan los activos.
        html=str(urllib.request.urlopen(url).read())
        posicion=html.find('Divisa en')
        divisa=html[posicion:posicion+13].split()[2]
        
        #Leo tabla de indices
        dfs = pd.read_html(url)
        dfs=dfs[0]
        
        #Comprobamos si Yahoo publica algún activo del índice
        if dfs.size>1:
            
            #Nos quedamos únicamente los activos que tengan cotización. 
            nombre_empresas=dfs.dropna().iloc[:,[0,1]]
            
            #Concatenamos en nuestro dataframe con todos los activo a devolver
            tabla_activos=pd.concat([tabla_activos, nombre_empresas]).reset_index(drop=True)
            
        else:
            
            print("El indice no tiene activos descargables")
            
    return tabla_activos

In [5]:
def datos_indices (tickers_de_indices,ventana):
    for benchmark in tickers_de_indices:
        print(benchmark+" descargando")
        
        # Obtengo las cookies de su web 
        cookieProcessor = urllib.request.HTTPCookieProcessor()
        opener = urllib.request.build_opener(cookieProcessor)
        
        # Una vez obtenida la cookie hago el res la url y obtengo todo el html para calcular crumb
        res=str(opener.open("https://es.finance.yahoo.com/quote/"+benchmark+"?ltr=1").read())
        
        # Buscamos y filtramos CrumbStore obteniendo el "pass" del crumb
        posicion=res.find('CrumbStore')
        crumb=res[posicion:posicion+35].split()
        inicio=crumb[0].find(':"')
        final=crumb[0].find('"}')
        crumb=crumb[0][inicio+2:final]
        
        # Convertimos las fechas al formato correspondiente y calculamos con la ventana el rango a obteer
        fecha_fin=int(round(time.time()))
        fecha_inicio=datetime.now()-timedelta(ventana)
        fecha_inicio=int(round(fecha_inicio.timestamp()))
        
        # Construimos la url con todo lo calculado anteriormente y guardamos los .csv en DF
        url="https://query1.finance.yahoo.com/v7/finance/download/"+benchmark+"?period1="+str(fecha_inicio)+"&period2="+str(fecha_fin)+"&interval=1d&events=history&crumb="+str(crumb)
        req = requests.get(url)
        data = StringIO(req.text)
        indices=pd.read_csv(data)
        
        #Guardamos los nombres de los df con el nombre del inice sin el %5 pasado en tickers_de_indices 
        globals()[benchmark.replace("%5", "").replace(".","_")]=indices
        print("Guardamos datos: " +benchmark+ " con nombre: " +benchmark.replace("%5", "").replace(".","_"))
        print("")

In [6]:
activos=activos_indices(tickers_de_indices)

In [7]:
datos_indices(activos.iloc[:,0],ventana)

0267.HK descargando
Guardamos datos: 0267.HK con nombre: 0267_HK

0011.HK descargando
Guardamos datos: 0011.HK con nombre: 0011_HK

1299.HK descargando
Guardamos datos: 1299.HK con nombre: 1299_HK

1044.HK descargando
Guardamos datos: 1044.HK con nombre: 1044_HK

2628.HK descargando
Guardamos datos: 2628.HK con nombre: 2628_HK

0003.HK descargando
Guardamos datos: 0003.HK con nombre: 0003_HK

2319.HK descargando
Guardamos datos: 2319.HK con nombre: 2319_HK

0012.HK descargando
Guardamos datos: 0012.HK con nombre: 0012_HK

0883.HK descargando
Guardamos datos: 0883.HK con nombre: 0883_HK

1038.HK descargando
Guardamos datos: 1038.HK con nombre: 1038_HK

2318.HK descargando
Guardamos datos: 2318.HK con nombre: 2318_HK

2018.HK descargando
Guardamos datos: 2018.HK con nombre: 2018_HK

1997.HK descargando
Guardamos datos: 1997.HK con nombre: 1997_HK

0101.HK descargando
Guardamos datos: 0101.HK con nombre: 0101_HK

1928.HK descargando


HTTPError: HTTP Error 503: Service Unavailable

### Otra manera de hacerlo

In [ ]:
import re
import io
from requests_html import HTMLSession
import pandas as pd
from datetime import datetime
from dateutil.relativedelta import relativedelta

In [ ]:
tickers_de_indices = ["%5EHSI", "%5EIBEX"]

In [ ]:
def obtener_datos_activo(activo, fecha_inicial, fecha_fin):

    # html de la web del activo
    request = HTMLSession().get(f"https://es.finance.yahoo.com/quote/{activo}?ltr=1")
    res = request.text

    # Obtenemos el CrumbStore
    pattern_crumbstore = r'\"CrumbStore\"\:{\"crumb\":\"(?P<crumb>.*?)\"}'
    crumb = re.search(pattern_crumbstore, res).groupdict().get('crumb')
    crumb = crumb.replace("\n", "")  # Algunas veces, el crumb tiene una función de escape (un salto de página).

    # Construimos la url para bajarnos los datos
    url = f"https://query1.finance.yahoo.com/v7/finance/download/{activo}?period1={fecha_inicial}&period2={fecha_fin}&interval=1d&events=history&crumb={crumb}"

    # info activo dataframe
    info_activo = pd.read_csv(io.StringIO(HTMLSession().get(url, cookies=request.cookies).text))
    return info_activo

In [ ]:
# Establecemos la ventana temporal
ventana = 60

In [ ]:
# Fechas inicio y fin
fecha_inicial = (datetime.now() - relativedelta(days=ventana)).strftime('%s')
fecha_fin = datetime.now().strftime('%s')

In [ ]:
for indice in tickers_de_indices:

    # Construimos la url que contiene la información de los activos de cada índice.
    web = f"https://es.finance.yahoo.com/quote/{indice}/components/"
    datos_web = HTMLSession().get(web).html

    # activos del indice
    activos = pd.read_html(datos_web.find('table', first=True).html)[0]

    if len(activos) > 1:  # Comprobamos si Yahoo publica algún activo del índice. Si la longitud es 1, no.

        # Ojo, queremos importar únicamente los activos que tengan cotización.
        activos_con_datos = activos.dropna(subset=['Último precio'])

        # simbolo de cada empresa del índice
        activos_a_importar = activos_con_datos['Símbolo'].to_list()

        # nombre de cada empresa del índice
        nombres_empresas = activos_con_datos['Nombre de la empresa'].to_list()

        # divisa de las empresas del índice
        divisa_empresas = (datos_web
                           .find('#Col1-0-Components-Proxy', first=True)
                           .find('span', first=True)
                           .text
                           .split(' ')[-1])
    else:
        print(f"El índice {indice} no tiene activos descargables")

    for activo in activos_a_importar:
        print(f'Descargando {activo}')

        info_activo = obtener_datos_activo(activo, fecha_inicial, fecha_fin)

        globals()[activo] = info_activo

        print(info_activo)

### Otra manera de hacerlo

In [1]:
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import json
from rcurl import get_curl

In [2]:
indices = ["%5EHSI", "%5EIBEX"]

ventana = 60
fecha_fin=int(time.time())
fecha_inicio=int(fecha_fin-86400*ventana)

for i in indices:
    url = f'https://es.finance.yahoo.com/quote/{i}/components?p={i}'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    texto=soup.find("div")
    sig=texto.findAll("span")
    linea=str(sig[7])
    divisa=linea[(linea.find("Divisa")+10):linea.find("</")]
    
    empresas=[]
    activos_a_importar=[]
    url = f'https://es.finance.yahoo.com/quote/{i}/components?p={i}'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    texto=soup.find("div")
    sig=texto.findAll("span")
    linea=str(sig[7])
    divisa=linea[(linea.find("Divisa")+10):linea.find("</")]

    
    url = f'https://es.finance.yahoo.com/quote/{i}/components?p={i}'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    tabla=soup.find('table')
    trs=tabla.findAll("tr")    
    for tr in trs[1:]:
        tds=tr.findAll('td')
        a=str(tds[1])
        b=str(tds[2])
        c=str(tds[0])
        nombre=a[(a.find('>')+1):(a.find('</'))]
        cotizacion=b[(b.find('>')+1):(b.find('</'))]
        c=c[(c.find("title")+7):]
        activo=c[:(c.find('">'))]
        if cotizacion != "" and cotizacion != "0,00":
            
            empresas.append(nombre)
            activos_a_importar.append(activo)
    
    for empresa in activos_a_importar:
        print(f'Descargando {empresa} que cotiza en {divisa}')

        url = f"https://es.finance.yahoo.com/quote/{empresa}?ltr=1"
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html')
        linea=str(soup)
        
        linea=linea[linea.find('CrumbStore'):(linea.find('CrumbStore')+50)]
        crumb=linea[(linea.find(':"')+2):linea.find('"}')]
        
        new_url=f'https://query1.finance.yahoo.com/v7/finance/download/{empresa}?period1={fecha_inicio}&period2={fecha_fin}&interval=1d&events=history&crumb={crumb}'
        
        contenido=requests.get(new_url)
        tabla = pd.read_csv(new_url, encoding='utf-8')
        globals()[empresa]=tabla
        print(tabla)      

Descargando 0669.HK que cotiza en HKD
          Date       Open       High        Low      Close  Adj Close  \
0   2020-05-04  57.000000  57.299999  56.400002  57.000000  56.513824   
1   2020-05-05  58.299999  62.150002  58.000000  62.049999  61.520748   
2   2020-05-06  62.950001  64.500000  61.099998  64.300003  63.751560   
3   2020-05-07  65.500000  65.500000  62.000000  63.950001  63.404545   
4   2020-05-08  65.500000  65.500000  63.700001  64.900002  64.346443   
5   2020-05-11  67.000000  67.750000  65.949997  66.550003  65.982368   
6   2020-05-12  65.500000  66.000000  64.500000  65.250000  64.693459   
7   2020-05-13  64.900002  66.349998  63.099998  65.750000  65.189194   
8   2020-05-14  65.599998  65.949997  63.000000  63.799999  63.255821   
9   2020-05-15  63.450001  64.650002  62.900002  64.000000  63.454117   
10  2020-05-18  63.549999  65.500000  63.349998  65.300003  64.743034   
11  2020-05-19  68.550003  68.699997  66.800003  68.000000  67.419998   
12  2020-05-2

KeyboardInterrupt: 

In [ ]:
globals()["SAN.MC"]

### Otra manera de hacerlo

In [ ]:
from datetime import datetime, timedelta
from time import mktime
import requests
import re
from lxml import html
from tqdm import tqdm
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
from bs4 import BeautifulSoup
import pandas as pd
tqdm.pandas()

In [ ]:
def get_index(index:str, start_date_unix:int,  end_date_unix:int):
    
    result = None
    try:
        url = f"https://es.finance.yahoo.com/quote/{index}?ltr=1"
        website = requests.get(url) 
        soup = BeautifulSoup(website.text)
        crumb = re.findall('"CrumbStore":{"crumb":"(.+?)"}', str(soup))[0]
        url2 = f"https://query1.finance.yahoo.com/v7/finance/download/{index}?period1={start_date_unix}&period2={end_date_unix}&interval=1d&events=history&crumb={crumb}"
        website = requests.get(url2).text.split('\n')
        table = [line.split(',') for line in website]
        result = pd.DataFrame(data=table[1:], columns=table[0])
        result.set_index('Date', inplace=True)
        result['Open'] = result['Open'].astype(float)
        result['High'] = result['High'].astype(float)
        result['Low'] = result['Low'].astype(float)
        result['Close'] = result['Close'].astype(float)
        result['Adj Close'] = result['Adj Close'].astype(float)
        result['Volume'] = result['Volume'].astype(float)
    
    except Exception as e:
        print(f'Index {index} could not be downloaded.')
        print(e)

    return result

In [ ]:
def getComponents(index:str):
    
    print(f'Downloading index {index}')
    
    result = []
    url = f"https://finance.yahoo.com/quote/{index}/components?p={index}"
    s = requests.Session()
    retries = Retry(total=3, backoff_factor=2, status_forcelist=[ 502, 503, 504 ])
    s.mount('http://', HTTPAdapter(max_retries=retries))
    website_url = s.get(url)
    
    tree = html.fromstring(website_url.content)
    currency = tree.xpath('//*[@id="quote-header-info"]/div[2]/div[1]/div[2]/span')[0].text
    result.append({'currency': currency[-3:]})
    
    soup = BeautifulSoup(website_url.text)
    table = soup.find('table', attrs={'class':"W(100%) M(0) BdB Bdc($finLightGray)"})
    try:
        tableData = table.tbody.find_all("tr")
        for register in tableData:
            symbol = register.find_all("td")[0].text
            name = register.find_all("td")[1].text
            volume = register.find_all("td")[3].text
            if len(volume) > 0:
                result.append({symbol: {'name': name}})
    
    except Exception as e:
        print(f'Component {index} could not be downloaded.')
        print(e)

    return result 

In [ ]:
ventana = 60

indices = ["%5EHSI", "%5EIBEX"]

end_date = datetime.today()
start_date = end_date.today() - timedelta(days=ventana)

end_date_unix = int(mktime(end_date.date().timetuple()))
start_date_unix = int(mktime(start_date.date().timetuple()))

In [ ]:
dictComponents = {index:getComponents(index) for index in tqdm(indices)}

In [ ]:
dictComponents

In [ ]:
for k,v in dictComponents.items():
    print(f'Downloading index {k}.')
    for index, elem in tqdm(enumerate(v)) :
        component = list(elem.keys())[0]
        if component != 'currency':
            data = get_index(index=component,
                             start_date_unix=start_date_unix,
                             end_date_unix=end_date_unix)
            dictComponents[k][index][component]['data'] = data

In [ ]:
dictComponents[indices[0]][:2]